In [1]:
%%html
<style> h1, h2, h3, h4 {font-weight: normal !important} </style>

In [2]:
# imports

import copy
import time
from pathlib import Path

import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import timm
import torch
from torchvision import models, datasets, transforms
from tqdm import tqdm

In [3]:
# config

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"using {device}")

# runs depending on platform

if Path("/kaggle/input").exists():
    platform = "kaggle"
    input_dir = Path("/kaggle/input/")
    save_dir = Path("/kaggle/output/")
else:
    platform = "local"
    input_dir = Path("./dataset/")
    save_dir = Path("./results/")
    
print(f"script running on {platform}")

random_seed = 13
print(f"using random seed: {random_seed}")

using cuda:0
script running on local
using random seed: 13


## Transforms

In [4]:
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomRotation(30),
        transforms.CenterCrop(224),
#         transforms.Resize(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#         transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    "valid": transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

## DataLoaders

In [5]:
list(input_dir.iterdir())

[PosixPath('dataset/train.csv'),
 PosixPath('dataset/test_images'),
 PosixPath('dataset/sample_submission.csv'),
 PosixPath('dataset/train_images')]

In [6]:
train_images_dir = input_dir/"train_images"
all_train_images = train_images_dir.glob("*/*")

test_images_dir = input_dir/"test_images"
all_test_images = test_images_dir.iterdir()

In [7]:
def create_df(images_gen, train=True):
    df = pd.DataFrame(list(images_gen), columns=["path"])
    if train:
        df["category"] = df.path.apply(lambda x: x.parent.name)
        le = LabelEncoder()
        df["encoded_label"] = le.fit_transform(df.category)
    return df

In [8]:
df = create_df(all_train_images)
df.head()

,path,category,encoded_label
0,dataset/train_images/bacterial_leaf_streak/107...,bacterial_leaf_streak,1
1,dataset/train_images/bacterial_leaf_streak/101...,bacterial_leaf_streak,1
2,dataset/train_images/bacterial_leaf_streak/104...,bacterial_leaf_streak,1
3,dataset/train_images/bacterial_leaf_streak/100...,bacterial_leaf_streak,1
4,dataset/train_images/bacterial_leaf_streak/104...,bacterial_leaf_streak,1


In [9]:
# split train and test
x_train, x_valid = train_test_split(df, test_size=0.2, shuffle=True, stratify=df.encoded_label, random_state=random_seed)
x_train.shape, x_valid.shape

((8325, 3), (2082, 3))

In [10]:
class MakeDataset(torch.utils.data.Dataset):
    
    def __init__(self, df, transform=None):
        self.df = df
        self.images = df.path.values
        self.labels = df.encoded_label.values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        image_path, label = self.images[idx], self.labels[idx]
        if self.transform:
            img = Image.open(image_path)
            img = self.transform(img)
        return img, label

In [11]:
trainset = MakeDataset(x_train, transform=data_transforms["train"])
validset = MakeDataset(x_valid, transform=data_transforms["valid"])

In [12]:
train_dl = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
valid_dl = torch.utils.data.DataLoader(validset, batch_size=16, shuffle=True, num_workers=4)

## Model build

In [13]:
# model = models.resnet18()
# num_ftrs = model.fc.in_features
# model.fc = torch.nn.Linear(num_ftrs, 10)
# model = model.to(device)

In [14]:
model = timm.create_model("levit_384", pretrained=True, num_classes=10)
model = model.to(device)

/home/tharun/.conda/envs/torch/lib/python3.10/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484803030/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [15]:
# define loss
criterion = torch.nn.CrossEntropyLoss()

In [16]:
# define optimizer
optim = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [17]:
# define lr_scheduler
lr_sch = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=0.1, patience=5)

### Train the model

In [18]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=100):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"epoch: {epoch}")
        print("-" * 10)

        running_loss = 0.0
        total_corrects = 0
        # train phase
        for i, (inputs, labels) in enumerate(train_dl):
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            total_corrects += torch.sum(predicted==labels.data)
        # print stats
        print(f"loss: {running_loss/len(x_train):.8f}")
        print(f"training acc: {total_corrects/len(x_train):.2f}")

        # validation phase
        total_corrects = 0
        running_loss = 0
        for i, (inputs, labels) in enumerate(valid_dl):
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, predicted = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                total_corrects += torch.sum(predicted==labels.data)
        epoch_valid_acc = total_corrects/len(x_valid)
        epoch_valid_loss = running_loss/len(x_valid)
        scheduler.step(epoch_valid_loss)
        
        print(f"valid acc: {epoch_valid_acc:.2f}")

        if epoch_valid_acc > best_acc:
            best_acc = epoch_valid_acc
            best_model_wts = copy.deepcopy(model.state_dict())
        
        print("\n")

    elapsed_time = time.time() - since
    print(f"total time taken: {elapsed_time/60:0f}m {elapsed_time%60:0f}s")
    print(f"best accuracy: {best_acc}")
    return model.load_state_dict(best_model_wts)

In [19]:
model = train_model(model, criterion, optim, lr_sch)

epoch: 0
----------
loss: 1.85536091
training acc: 0.37
valid acc: 0.40


epoch: 1
----------
loss: 1.28523064
training acc: 0.57
valid acc: 0.52


epoch: 2
----------
loss: 0.99486742
training acc: 0.67
valid acc: 0.57


epoch: 3
----------
loss: 0.80823612
training acc: 0.73
valid acc: 0.64


epoch: 4
----------
loss: 0.66237168
training acc: 0.79
valid acc: 0.63


epoch: 5
----------
loss: 0.54387833
training acc: 0.83
valid acc: 0.65


epoch: 6
----------
loss: 0.48159271
training acc: 0.84
valid acc: 0.66


epoch: 7
----------
loss: 0.39446288
training acc: 0.87
valid acc: 0.68


epoch: 8
----------
loss: 0.34267245
training acc: 0.89
valid acc: 0.69


epoch: 9
----------
loss: 0.32254438
training acc: 0.89
valid acc: 0.74


epoch: 10
----------
loss: 0.26956554
training acc: 0.91
valid acc: 0.72


epoch: 11
----------
loss: 0.24496070
training acc: 0.92
valid acc: 0.71


epoch: 12
----------
loss: 0.20751811
training acc: 0.93
valid acc: 0.73


epoch: 13
----------
loss: 0.192022

## Inference

In [20]:
test_df = create_df(all_test_images, train=False)
test_df.head()

,path
0,dataset/test_images/200761.jpg
1,dataset/test_images/201036.jpg
2,dataset/test_images/201042.jpg
3,dataset/test_images/203280.jpg
4,dataset/test_images/200126.jpg


In [26]:
test_df["label"] = None
model.eval()

for key, value in test_df.path.items():
    img = Image.open(value)
    img = data_transforms["valid"](img)
    img = img.unsqueeze(0).to(device)
    prediction = model(img).argmax().item()
    label = le.inverse_transform([prediction])[0]
    test_df.loc[key, "label"] = label

test_df.head()

AttributeError: '_IncompatibleKeys' object has no attribute 'eval'

In [ ]:
subm_df = test_df.copy(deep=True)
subm_df["image_id"] = subm_df.path.apply(lambda x: x.name)

subm_df.drop("path", axis=1, inplace=True)
subm_df.to_csv(save_dir/"levit384.csv", index=False)

In [ ]:
subm_df.label.value_counts()